### Information Retrieval   |    Assignment 1-B

Step 1:

In [1]:
import pandas as pd
import numpy as np
import sys
import os
import random as rnd

In [43]:
df_raw = pd.read_table('YandexRelPredChallenge.txt', delimiter="\t",index_col=None, 
                 header=0,names=['SessionID', 'TimePassed','TypeOfAction', 'QueryID','RegionID', 'ListOfURLs', 'URLID',
                                 '1','2','3','4','5',"6",'7','8'])
df_raw

In [2]:
# Open file    
fileHandler = open ('YandexRelPredChallenge.txt', "r")
# Get list of all lines in file
listOfLines = fileHandler.readlines()
# Close file 
fileHandler.close()
#Empty frame to fill in
index=[x for x in range(0,100000)]
df = pd.DataFrame(index=index, columns=['SessionID', 'TimePassed',
                 'TypeOfAction', 'QueryID','RegionID', 'ListOfURLs', 'URLID'])
df.name='data'

# Iterate over the lines and fill in the dataframe
for ind, line in zip(index, listOfLines):
#     print('index')
    line = line.strip().split()
    if line[2]=='Q':

        df.loc[ind] = pd.Series({'SessionID':line[0], 'TimePassed':line[1], 'TypeOfAction':line[2],
                                     'QueryID':line[3],'RegionID':line[4], 'ListOfURLs':line[5:], 'URLID':np.nan})
    else:
        df.loc[ind] = pd.Series({'SessionID':line[0], 'TimePassed':line[1], 'TypeOfAction':line[2],
                                     'QueryID':np.nan,'RegionID':np.nan, 'ListOfURLs':np.nan, 'URLID':line[3:]})

In [3]:
# Getting the types right. 
df.SessionID = df.SessionID.astype(int)
df.TimePassed = df.TimePassed.astype(int)
df.TypeOfAction = df.TypeOfAction.astype(str)
df.RegionID = df.RegionID.astype(float)
df.TimePassed = df.TimePassed.astype(float)
def parse_ulrid(x):
    if type(x)==list:
        return x[0]
    else:
        return x
df.URLID = df.URLID.apply(parse_ulrid)
df.URLID = df.URLID.astype(float)

Create new relevance:

So lets say that our experimental system is what Yandex return as a list now. Production or the system that is going to 'suck' (sort of a dumb system that we are going to compare against) will be constructed as shuffling each ranking for a query. 

Now, other approach, probably the better one: take the initial order as the 'benchmark' create only list of 0,1 (relevnace scores) for each system.
    - rember that we are interested only in three [3] first results.

In [34]:
# Create the relevance score ranking (those tables = that we called 'runs')
def relevance_list(l,acc=0.5, res_length=3):
    """ Control the 'accuracy'/'performance' with acc param. Only lists of length 3. """
    if type(l)==list:
        res = list()
        for x in l:
            if rnd.random()<acc:
                res.append((x, 1))
            else:
                res.append((x, 0))
        return rnd.sample(res, res_length)
    else:
        return l

df['id_rel_e'] = df.ListOfURLs.apply(relevance_list, acc=0.7) 
df['id_rel_p'] = df.ListOfURLs.apply(relevance_list) 

In [35]:
def extract_rel(l):
    if type(l) == list:
        res = list()
        for tup in l:
            res.append(tup[1])
        return res
    else:
        return l
def extract_id(l):
    if type(l) == list:
        res = list()
        for tup in l:
            res.append(tup[0])
        return res
    else:
        return l
df['rel_e'] = df.id_rel_e.apply(extract_rel)
df['rel_p'] = df.id_rel_p.apply(extract_rel)
df['id_e'] = df.id_rel_e.apply(extract_id)
df['id_p'] = df.id_rel_p.apply(extract_id)

Step 2: Calculate ERR

see the formuala for ERR at: http://olivier.chapelle.cc/pub/err.pdf

In [36]:
def ERR(l):
    """ List or relevance scores. """ 
    if type(l)==list:
        theta = [ (2**i - 1)/(2) for i in [0,1]]
        err=0
        for rank in range(len(l)):
            prod = 1
            for idx in range(rank):
                prod*=(1-theta[l[idx]])
            prod*=(1/(rank+1))*theta[l[rank]]
            err+=prod
        return err
    else:
        return l
df['err_e'] = df.rel_e.apply(ERR)
df['err_p'] = df.rel_p.apply(ERR)

Get the deltas

In [40]:
# Test to see if ERR calculation is implemented correctly and if E outperfroms P 
import math 
df['diff_e_p'] = df.err_e - df.err_p
def count_t(x):
    if not math.isnan(x):
        if x>0:
            return 1
        else:
            return 0 
    else:
        return x 
df['temp_count'] = df.diff_e_p.apply(count_t)
df.loc[:,['diff_e_p', 'temp_count']]
df.temp_count.value_counts()
df.drop(columns=['diff_e_p','temp_count'],inplace=True)

### Step 4.1: Estimating the params of click models: Random Click Model

In [179]:
# Get all the doc ids
data = df.drop(columns=['id_rel_e', 'id_rel_p', 'rel_e', 'rel_p', 'id_e',
       'id_p', 'err_e', 'err_p'])
data.QueryID = data.QueryID.astype(float)
list_doc_ids_raw = data.ListOfURLs[df.ListOfURLs.isnull()!=True].values
res = list()
for i in list_doc_ids_raw:
    res += i
all_doc_ids = res.copy()
# Get all the clicks
list_clicks_ids = data.URLID[df.URLID.isnull()!=True].values

# rho = probability of a doc to be clicked.
rho = len(list_clicks_id)/len(all_doc_ids)
print(rho)

0.13445559411047547


### Step 4.2: Estimating the params of click models: Posistion-based Model

Perform EM steps (update assignments ; update params):
    
    1) initialize alpha and gamma at random 
    2) For each time step:
        - Expecation: do not really have to do this step [?], what we want to see at the end: P(C|u) 
            - calculate P(C|u) for each u (u is from 1 to 3 = positions in the shown list)
            - " P(C|u) = P(A_uq|E,u)P(E|u) "
                
        - Maximization: update alpha and gamma params
        


In [220]:
# Create the tables with all the params
n_docs = 3
unique_queries = data.QueryID[data.QueryID.isnull()!=True].astype(int).unique()
n_queries = len(unique_queries) 
alphas = np.random.rand(n_docs)
gammas = np.random.rand(n_docs*n_queries).reshape((n_docs,n_queries))

In [357]:
# number of sessions
sessions = data.SessionID.unique()

In [381]:
# this code will give an error at the very last session
for session in sessions:
    print('sesssion', session)
    data_session = data[data.SessionID==session]
    click_ids =  data_session.URLID[data_session.URLID.isnull()!=True].values
    query_ids = data_session.QueryID[data_session.QueryID.isnull()!=True].values
    for query_id in query_ids:
        first_3_docs = data_session.ListOfURLs[data.QueryID==query_id].iloc[0]
        first_3_docs = np.array( list(map(int,first_3_docs))[:3] )
        # take index in the current data_session
        query_id_ind = data_session.QueryID[data_session.QueryID==query_id].index[0]
        flag = not math.isnan(data.URLID.iloc[query_id_ind+1])
        # if the next entry is click
        if flag:
            click_log = np.array([0,0,0])
            query_id_ind+=1
            while flag==1:
                click_urlid = data.URLID.iloc[query_id_ind]
                # if click_urlid in first_3_docs then click is 1 on that position.
                doc_u_id = np.where(first_3_docs==click_urlid)
                click_log[doc_u_id]=1
                # increase the count
                query_id_ind +=1
                flag = not math.isnan(data.URLID.iloc[query_id_ind])
            # do updates 
        else:
            click_log = np.array([0,0,0])
            # do updates

sesssion 0
sesssion 1
sesssion 2
sesssion 3
sesssion 4
sesssion 5
sesssion 6
sesssion 7
sesssion 8
sesssion 9
sesssion 10
sesssion 11
sesssion 12
sesssion 13
sesssion 14
sesssion 15
sesssion 16
sesssion 17
sesssion 19
sesssion 20
sesssion 22
sesssion 23
sesssion 24
sesssion 25
sesssion 26
sesssion 27
sesssion 28
sesssion 29
sesssion 30
sesssion 31
sesssion 32
sesssion 33
sesssion 34
sesssion 35
sesssion 36
sesssion 37
sesssion 38
sesssion 39
sesssion 40
sesssion 41
sesssion 42
sesssion 43
sesssion 45
sesssion 46
sesssion 47
sesssion 48
sesssion 49
sesssion 50
sesssion 52
sesssion 54
sesssion 55
sesssion 56
sesssion 57
sesssion 58
sesssion 59
sesssion 60
sesssion 61
sesssion 62
sesssion 64
sesssion 65
sesssion 66
sesssion 67
sesssion 68
sesssion 69
sesssion 70
sesssion 71
sesssion 72
sesssion 73
sesssion 75
sesssion 76
sesssion 77
sesssion 79
sesssion 80
sesssion 81
sesssion 82
sesssion 83
sesssion 85
sesssion 87
sesssion 88
sesssion 89
sesssion 90
sesssion 92
sesssion 93
sesssion 94
se

sesssion 759
sesssion 760
sesssion 762
sesssion 763
sesssion 764
sesssion 765
sesssion 766
sesssion 767
sesssion 768
sesssion 769
sesssion 770
sesssion 771
sesssion 772
sesssion 773
sesssion 776
sesssion 777
sesssion 778
sesssion 779
sesssion 780
sesssion 781
sesssion 783
sesssion 784
sesssion 785
sesssion 788
sesssion 789
sesssion 790
sesssion 791
sesssion 792
sesssion 793
sesssion 795
sesssion 796
sesssion 797
sesssion 798
sesssion 799
sesssion 800
sesssion 801
sesssion 802
sesssion 803
sesssion 804
sesssion 805
sesssion 809
sesssion 810
sesssion 811
sesssion 812
sesssion 813
sesssion 814
sesssion 815
sesssion 816
sesssion 817
sesssion 819
sesssion 820
sesssion 821
sesssion 822
sesssion 823
sesssion 824
sesssion 825
sesssion 826
sesssion 827
sesssion 828
sesssion 831
sesssion 832
sesssion 834
sesssion 835
sesssion 836
sesssion 837
sesssion 839
sesssion 840
sesssion 841
sesssion 842
sesssion 844
sesssion 845
sesssion 846
sesssion 847
sesssion 849
sesssion 850
sesssion 851
sesssion 852

sesssion 1492
sesssion 1494
sesssion 1496
sesssion 1497
sesssion 1499
sesssion 1500
sesssion 1502
sesssion 1503
sesssion 1504
sesssion 1505
sesssion 1506
sesssion 1507
sesssion 1508
sesssion 1509
sesssion 1511
sesssion 1512
sesssion 1513
sesssion 1514
sesssion 1515
sesssion 1516
sesssion 1517
sesssion 1518
sesssion 1520
sesssion 1521
sesssion 1522
sesssion 1523
sesssion 1525
sesssion 1526
sesssion 1527
sesssion 1529
sesssion 1530
sesssion 1531
sesssion 1532
sesssion 1533
sesssion 1534
sesssion 1535
sesssion 1536
sesssion 1537
sesssion 1538
sesssion 1539
sesssion 1540
sesssion 1541
sesssion 1542
sesssion 1543
sesssion 1544
sesssion 1546
sesssion 1548
sesssion 1549
sesssion 1550
sesssion 1551
sesssion 1552
sesssion 1554
sesssion 1555
sesssion 1556
sesssion 1557
sesssion 1558
sesssion 1559
sesssion 1560
sesssion 1561
sesssion 1563
sesssion 1564
sesssion 1566
sesssion 1570
sesssion 1571
sesssion 1572
sesssion 1573
sesssion 1574
sesssion 1575
sesssion 1576
sesssion 1577
sesssion 1578
sesssi

sesssion 2220
sesssion 2221
sesssion 2222
sesssion 2223
sesssion 2224
sesssion 2225
sesssion 2226
sesssion 2228
sesssion 2229
sesssion 2230
sesssion 2231
sesssion 2234
sesssion 2235
sesssion 2237
sesssion 2238
sesssion 2239
sesssion 2240
sesssion 2241
sesssion 2242
sesssion 2243
sesssion 2244
sesssion 2245
sesssion 2246
sesssion 2248
sesssion 2249
sesssion 2250
sesssion 2251
sesssion 2252
sesssion 2253
sesssion 2254
sesssion 2255
sesssion 2256
sesssion 2257
sesssion 2259
sesssion 2260
sesssion 2261
sesssion 2262
sesssion 2263
sesssion 2266
sesssion 2267
sesssion 2268
sesssion 2269
sesssion 2270
sesssion 2271
sesssion 2272
sesssion 2273
sesssion 2276
sesssion 2277
sesssion 2278
sesssion 2279
sesssion 2280
sesssion 2281
sesssion 2282
sesssion 2283
sesssion 2286
sesssion 2288
sesssion 2289
sesssion 2290
sesssion 2291
sesssion 2292
sesssion 2293
sesssion 2294
sesssion 2295
sesssion 2296
sesssion 2297
sesssion 2299
sesssion 2300
sesssion 2301
sesssion 2302
sesssion 2303
sesssion 2306
sesssi

sesssion 2919
sesssion 2920
sesssion 2921
sesssion 2923
sesssion 2924
sesssion 2925
sesssion 2926
sesssion 2927
sesssion 2928
sesssion 2929
sesssion 2930
sesssion 2931
sesssion 2933
sesssion 2934
sesssion 2935
sesssion 2936
sesssion 2937
sesssion 2939
sesssion 2940
sesssion 2941
sesssion 2942
sesssion 2943
sesssion 2944
sesssion 2945
sesssion 2946
sesssion 2947
sesssion 2948
sesssion 2949
sesssion 2951
sesssion 2952
sesssion 2953
sesssion 2954
sesssion 2955
sesssion 2956
sesssion 2957
sesssion 2958
sesssion 2959
sesssion 2960
sesssion 2962
sesssion 2963
sesssion 2964
sesssion 2965
sesssion 2966
sesssion 2968
sesssion 2970
sesssion 2971
sesssion 2972
sesssion 2973
sesssion 2976
sesssion 2978
sesssion 2979
sesssion 2980
sesssion 2981
sesssion 2982
sesssion 2983
sesssion 2984
sesssion 2985
sesssion 2986
sesssion 2987
sesssion 2988
sesssion 2989
sesssion 2990
sesssion 2991
sesssion 2993
sesssion 2994
sesssion 2995
sesssion 2997
sesssion 2998
sesssion 2999
sesssion 3000
sesssion 3002
sesssi

...

sesssion 11303
sesssion 11304
sesssion 11305
sesssion 11306
sesssion 11307
sesssion 11309
sesssion 11312
sesssion 11313
sesssion 11314
sesssion 11315
sesssion 11316
sesssion 11318
sesssion 11319
sesssion 11320
sesssion 11321
sesssion 11322
sesssion 11323
sesssion 11324
sesssion 11325
sesssion 11326
sesssion 11328
sesssion 11330
sesssion 11331
sesssion 11332
sesssion 11334
sesssion 11335
sesssion 11337
sesssion 11338
sesssion 11339
sesssion 11340
sesssion 11341
sesssion 11342
sesssion 11343
sesssion 11344
sesssion 11345
sesssion 11346
sesssion 11348
sesssion 11349
sesssion 11350
sesssion 11351
sesssion 11353
sesssion 11354
sesssion 11355
sesssion 11356
sesssion 11357
sesssion 11358
sesssion 11359
sesssion 11360
sesssion 11361
sesssion 11362
sesssion 11363
sesssion 11364
sesssion 11365
sesssion 11366
sesssion 11368
sesssion 11369
sesssion 11370
sesssion 11371
sesssion 11372
sesssion 11373
sesssion 11374
sesssion 11376
sesssion 11377
sesssion 11378
sesssion 11379
sesssion 11380
sesssion 1

sesssion 11970
sesssion 11971
sesssion 11972
sesssion 11973
sesssion 11974
sesssion 11975
sesssion 11977
sesssion 11978
sesssion 11979
sesssion 11980
sesssion 11981
sesssion 11982
sesssion 11983
sesssion 11984
sesssion 11985
sesssion 11986
sesssion 11987
sesssion 11989
sesssion 11990
sesssion 11991
sesssion 11994
sesssion 11995
sesssion 11997
sesssion 11998
sesssion 11999
sesssion 12000
sesssion 12001
sesssion 12002
sesssion 12003
sesssion 12004
sesssion 12005
sesssion 12006
sesssion 12008
sesssion 12009
sesssion 12010
sesssion 12012
sesssion 12014
sesssion 12018
sesssion 12019
sesssion 12020
sesssion 12022
sesssion 12023
sesssion 12024
sesssion 12025
sesssion 12026
sesssion 12027
sesssion 12028
sesssion 12029
sesssion 12030
sesssion 12031
sesssion 12032
sesssion 12033
sesssion 12035
sesssion 12036
sesssion 12037
sesssion 12038
sesssion 12039
sesssion 12040
sesssion 12041
sesssion 12042
sesssion 12043
sesssion 12044
sesssion 12046
sesssion 12047
sesssion 12048
sesssion 12049
sesssion 1

sesssion 12642
sesssion 12643
sesssion 12644
sesssion 12645
sesssion 12646
sesssion 12647
sesssion 12648
sesssion 12649
sesssion 12651
sesssion 12652
sesssion 12653
sesssion 12654
sesssion 12656
sesssion 12657
sesssion 12658
sesssion 12659
sesssion 12660
sesssion 12661
sesssion 12663
sesssion 12664
sesssion 12665
sesssion 12666
sesssion 12668
sesssion 12669
sesssion 12670
sesssion 12671
sesssion 12672
sesssion 12674
sesssion 12675
sesssion 12676
sesssion 12677
sesssion 12678
sesssion 12679
sesssion 12680
sesssion 12681
sesssion 12682
sesssion 12683
sesssion 12684
sesssion 12685
sesssion 12686
sesssion 12687
sesssion 12689
sesssion 12690
sesssion 12691
sesssion 12692
sesssion 12693
sesssion 12694
sesssion 12695
sesssion 12696
sesssion 12697
sesssion 12700
sesssion 12701
sesssion 12702
sesssion 12703
sesssion 12704
sesssion 12705
sesssion 12706
sesssion 12707
sesssion 12708
sesssion 12709
sesssion 12710
sesssion 12711
sesssion 12713
sesssion 12714
sesssion 12715
sesssion 12716
sesssion 1

sesssion 13326
sesssion 13327
sesssion 13328
sesssion 13330
sesssion 13331
sesssion 13332
sesssion 13333
sesssion 13334
sesssion 13335
sesssion 13337
sesssion 13338
sesssion 13339
sesssion 13342
sesssion 13343
sesssion 13344
sesssion 13345
sesssion 13346
sesssion 13348
sesssion 13349
sesssion 13350
sesssion 13351
sesssion 13353
sesssion 13354
sesssion 13355
sesssion 13356
sesssion 13357
sesssion 13358
sesssion 13359
sesssion 13360
sesssion 13361
sesssion 13363
sesssion 13365
sesssion 13366
sesssion 13367
sesssion 13368
sesssion 13369
sesssion 13370
sesssion 13372
sesssion 13373
sesssion 13374
sesssion 13375
sesssion 13376
sesssion 13377
sesssion 13378
sesssion 13379
sesssion 13380
sesssion 13381
sesssion 13382
sesssion 13383
sesssion 13385
sesssion 13388
sesssion 13389
sesssion 13390
sesssion 13391
sesssion 13392
sesssion 13393
sesssion 13394
sesssion 13395
sesssion 13396
sesssion 13397
sesssion 13398
sesssion 13399
sesssion 13400
sesssion 13401
sesssion 13402
sesssion 13403
sesssion 1

IndexError: single positional indexer is out-of-bounds

In [382]:
sessions[-1]

13869

In [238]:
# Write the EM algo,
# if alphas are hardcoded, then  :
    # Get sessions with Query(Q) and doc(u) => will do implicit, coz can use pandas slicing
for i in range(0,100):
    for gamma_ind in range(len(gammas)):
        summ = 0
        for session_i in sessions:
            temp_sum = 
        temp_gamma = 1/len(sessions)*summ
#     gammas = 

In [250]:
arr = list(test.groupby(by="SessionID").count().index)
arr

[3]